In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import scipy.signal as ss
import seaborn as sns
import numpy as np
from skimage.io import imread
from glob import glob
import os
import h5py

### Load Ben's processed data and apply filter
---
#### Download the following files from /Volumes/genie/BenArthur/GETI-screen/kaspars-method-v2 to sumPath
- summary-good-1AP.csv
- summary-good-20AP.csv

In [2]:
sumPath = '/Users/zhengj10/Desktop/iGluSnFR'

In [3]:
stim = '1AP'
df_ben = pd.read_csv(sumPath+'/good_%s.csv' %stim)
print(df_ben.shape)
df_ben.construct.unique().shape

(11783, 32)


(9270,)

In [4]:
# df_ben['transfect/plate'].unique()
# df_ben.sort_values(by='transfect/plate',ascending=False).to_excel('/Users/zhengj10/Desktop/test.xlsx')
# df_ben.sort_values(by='transfect/plate',ascending=False)
# df_ben[df_ben['construct'].str.contains('6428')]

#### Extract params from .csv file with explicit param names

In [5]:
df_plot = pd.DataFrame()
df_ben.columns

Index(['construct', 'transfect/plate', 'F<sub>0</sub>', 'pval 1-tailed',
       'pval 2-tailed', 'unnormalized', '&#916;F/F<sub>0</sub>*&#8730;F0',
       'pval 1-tailed.1', 'pval 2-tailed.1', 'unnormalized.1',
       '&Delta;F/F<sub>0</sub>', 'pval 1-tailed.2', 'pval 2-tailed.2',
       'unnormalized.2', 'T<sub>off</sub>', 'pval 1-tailed.3',
       'pval 2-tailed.3', 'unnormalized.3', 'T<sub>on</sub>',
       'pval 1-tailed.4', 'pval 2-tailed.4', 'unnormalized.4', 'fast &Delta;F',
       'pval 1-tailed.5', 'pval 2-tailed.5', 'unnormalized.5',
       '#pixels<sub>&nbsp;</sub>', '#wells<sub>&nbsp;</sub>',
       'ctrl #pix<sub>&nbsp;</sub>', 'ctrl pdi<sub>&nbsp;</sub>',
       'ctrl &Delta;F/F<sub>0</sub>', 'isbad?'],
      dtype='object')

In [6]:
df_plot['date/plate'] = df_ben['transfect/plate']
df_plot['construct'] = df_ben['construct']
df_plot['no_pixels'] = df_ben['#pixels<sub>&nbsp;</sub>']
df_plot['thre'] = 1+(df_ben['ctrl pdi<sub>&nbsp;</sub>']/100.0)
df_plot['ctr_no_pixels'] = df_ben['ctrl #pix<sub>&nbsp;</sub>']

# Normalized data
# df_plot['SNR'] = df_ben['SNR<sub></sub>'] # SNR data unavailable in data re-analyzed in Jan.2023
df_plot['dFF'] = df_ben['&Delta;F/F<sub>0</sub>']
df_plot['fast_dF'] = df_ben['fast &Delta;F']
df_plot['fast_dFF'] = df_ben['fast &Delta;F']/df_ben['F<sub>0</sub>']
df_plot['F0'] = df_ben['F<sub>0</sub>']
df_plot['tau_on'] = df_ben['T<sub>on</sub>']
df_plot['tau_off'] = df_ben['T<sub>off</sub>']
df_plot['dFF_Tau_on_ratio'] = df_ben['&Delta;F/F<sub>0</sub>']/df_ben['T<sub>on</sub>']

# Unnormalized data
# df_plot['SNR_unnorm'] = df_ben['unnormalized'] # SNR data unavailable in data re-analyzed in Jan.2023
df_plot['dFF_unnorm'] = df_ben['unnormalized.2']
df_plot['fast_dF_unnorm'] = df_ben['unnormalized.5']
df_plot['fast_dFF_unnorm'] = df_ben['unnormalized.5']/df_ben['unnormalized']
df_plot['F0_unnorm'] = df_ben['unnormalized']
df_plot['tau_on_unnorm'] = df_ben['unnormalized.4']
df_plot['tau_off_unnorm'] = df_ben['unnormalized.3']

# Control data
# df_plot['ctr_SNR'] = df_ben['unnormalized']/df_ben['SNR<sub></sub>'] # SNR data unavailable in data re-analyzed in Jan.2023
df_plot['ctr_dFF'] = df_ben['ctrl &Delta;F/F<sub>0</sub>']
df_plot['ctr_fast_dF'] = df_ben['unnormalized.5']/df_ben['fast &Delta;F']
df_plot['ctr_fast_dFF'] = (df_ben['unnormalized.5']/df_ben['fast &Delta;F'])/(df_ben['unnormalized']/df_ben['F<sub>0</sub>'])
df_plot['ctr_F0'] = df_ben['unnormalized']/df_ben['F<sub>0</sub>']
df_plot['ctr_tau_on'] = df_ben['unnormalized.4']/df_ben['T<sub>on</sub>']
df_plot['ctr_tau_off'] = df_ben['unnormalized.5']/df_ben['T<sub>off</sub>']
df_plot['ctr_dFF_Tau_on_ratio'] = df_ben['ctrl &Delta;F/F<sub>0</sub>']/df_ben['unnormalized.4']/df_ben['T<sub>on</sub>']

df_plot.shape

(11783, 25)

In [7]:
# df_plot[df_plot['date/plate']=='20221128_iGluSnFR/P11a-20221114_iGluSnFR'].construct.unique()

#### Add another column of construct id

In [8]:
C = []
C_dot = []
for i,r in df_plot.iterrows():
    c = r['construct'].split('.')[1]+'.'+r['construct'].split('.')[2]
    c = c.split(';')[0]
    c_dot = c.replace('.','dot')
    C.append(c)
    C_dot.append(c_dot)
df_plot['construct_id'] = C
df_plot['construct_id_dot'] = C_dot
df_plot = df_plot.sort_values(by=['date/plate','construct_id'],ascending=True,ignore_index=True)

In [9]:
print(df_plot.loc[9156,'construct_id']) # Last single: 633.6412
df_plot.loc[9157,'construct_id'] # First combo: 7887 is the 1st combo

633.6412


'633.7887'

#### Create seperate dateframe for single, combo and mixed (combo and their single parents)

In [10]:
idx = 9156 # last single mutant after sorting data by ['date/plate','construct_id'] 633.6412
df_single = df_plot.loc[:idx,:]
df_combo = df_plot.loc[idx+1:,:] # 7883 is the start of iGluSnFR-combo according to GT (7883 not in constructs)
df_combo = df_combo[~df_combo['construct_id'].isin(df_single['construct_id'].unique())]

# List of date/plate (l) for combo and single parents
l = ['20220822_iGluSnFR/P',
    '20220927_iGluSnFR/P',
    '20221004_iGluSnFR/P',
    '20221011_iGluSnFR/P',
    '20221024_iGluSnFR/P']

df_t = pd.DataFrame()
for d in l:
    df_t = pd.concat([df_t,df_plot[df_plot['date/plate'].str.contains(d)]],axis=0)
df_comp_single_vs_combo = df_plot[df_plot['date/plate'].isin(df_t['date/plate'].unique())]

In [11]:
df_single.shape
# df_combo.shape
df_single.construct.unique().shape
df_combo.construct.unique().shape

(1641,)

#### Save each dataframe to pickle file for loading with plotting script

In [12]:
df_single.to_pickle(sumPath+'/iGluSnFR_screen_single_%s.pkl' %stim)
df_combo.to_pickle(sumPath+'/iGluSnFR_screen_combo_%s.pkl' %stim)
df_comp_single_vs_combo.to_pickle(sumPath+'/iGluSnFR_screen_comp_single_vs_combo_%s.pkl' %stim)

In [ ]:
df_plot = df_plot.dropna()
df_plot = df_plot[df_plot['no_pixels']>1e4]
df_plot.shape

In [ ]:
sns.set_style(style='whitegrid')
fig = plt.figure(figsize=(13,7.5))
plt.rc('font', size=13)
ax = plt.subplot(111)

scale = 20 # for combo mutants
all_sensors = plt.scatter(df_plot['fast_dF'],df_plot['tau_off'],c=df_plot['no_pixels'],s=df_plot['F0']*scale,
                          cmap="jet")

plt.clim(a,b)
cbar = plt.colorbar(all_sensors,aspect=50)
cbar.ax.set_ylabel('No. pixels',rotation=90,size=14)

plt.legend(*all_sensors.legend_elements("sizes", num = 5,func = lambda x: x/scale),title='Norm. F0')
ax.set_xlabel('Norm. fast \u0394F (1AP)',size=14)
ax.set_ylabel('Norm. tau off (1AP)',size=14)


plt.tight_layout()
savePath = '/Volumes/genie/Presentations/Jihong/GENIE production/221220/iGluSnFR_screening'
if not os.path.exists(savePath):
    os.makedirs(savePath)
# plt.savefig(savePath+'/Kerr.svg',format='svg')
# plt.savefig('/Users/zhengj10/Desktop/20AP.pdf',format='pdf')
plt.show()

In [ ]:
a = np.quantile(df_plot['no_pixels'],0.1)
b = np.quantile(df_plot['no_pixels'],0.9)